I used an ultrasonic sensor (Grove Ultrasonic Ranger V2.0), an Arduino Uno, and a Dragino LoRa shield V1.4. I modified the original code slightly. The code runs in the Ardunino IDE and sends data evey 60 seconds to thethingsnetwork.org console.

In [ ]:
//==>Copyright (c) 2015 Thomas Telkamp and Matthijs Kooijman

#include <lmic.h>
#include <hal/hal.h>
#include <SPI.h>
#include <Wire.h>
#include "Ultrasonic.h"


//==>Enter LoRa Keys from the thingsnetwork
u1_t NWKSKEY[16] = { 0x0A, 0xC6, 0xE6, 0x54, 0x7E, 0x6D, 0xB2, 0xA2, 0x42, 0xB9, 0x02, 0xFB, 0x88, 0xAF, 0x3F, 0xBC };
u1_t APPSKEY[16] = { 0x8D, 0x66, 0x94, 0x5B, 0xDB, 0xC7, 0x1A, 0xA5, 0x35, 0xE6, 0xF5, 0x39, 0x45, 0x26, 0xB6, 0x10 };
static const u4_t DEVADDR = 0x26011801 ; // <-- Change this address for every node!

void os_getArtEui (u1_t* buf) { }
void os_getDevEui (u1_t* buf) { }
void os_getDevKey (u1_t* buf) { }



Ultrasonic ultrasonic(4); //wired to Digital 4 on Arduino Uno

static osjob_t sendjob;

const unsigned TX_INTERVAL = 60;


//==>Dragino Pin mapping
const lmic_pinmap lmic_pins = {
    .nss = 10,
    .rxtx = LMIC_UNUSED_PIN,
    .rst = 9,
    .dio = {2, 6, 7},
};



//LMIC Events

void onEvent (ev_t ev) {
     if (ev == EV_TXCOMPLETE) os_setTimedCallback(&sendjob, os_getTime()+sec2osticks(TX_INTERVAL), do_send);
}


//==>Job and Ultrasonic Measurement
void do_send(osjob_t* j){
    // Check if there is not a current TX/RX job running
    if (LMIC.opmode & OP_TXRXPEND) {
        Serial.println(F("OP_TXRXPEND, not sending"));
    } else {
        // Prepare upstream data transmission at the next possible time.
               
        byte Range = ultrasonic.MeasureInCentimeters();   // two measurements should keep an interval
        Serial.println("The distance to obstacles in front is: ");
        Serial.print(Range);   //0~400cm
        Serial.println(" cm");
        delay(60000);

        byte payload[2];
        payload[0] = highByte(Range);
        payload[1] = lowByte(Range);
         
        LMIC_setTxData2(1, payload, sizeof(payload), 0);
        Serial.println(F("Packet queued"));
    }
    // Next TX is scheduled after TX_COMPLETE event.
}

//==>Arduino Uno setup and loop
void setup() {
  
    Serial.begin(115200);
    Serial.println(F("Starting"));
  
    os_init();
    LMIC_reset();
    LMIC_setSession (0x1, DEVADDR, NWKSKEY, APPSKEY);
      
    LMIC_setupChannel(0, 868100000, DR_RANGE_MAP(DR_SF12, DR_SF7),  BAND_CENTI);      // g-band
    LMIC_setupChannel(1, 868300000, DR_RANGE_MAP(DR_SF12, DR_SF7B), BAND_CENTI);      // g-band
    LMIC_setupChannel(2, 868500000, DR_RANGE_MAP(DR_SF12, DR_SF7),  BAND_CENTI);      // g-band
    LMIC_setupChannel(3, 867100000, DR_RANGE_MAP(DR_SF12, DR_SF7),  BAND_CENTI);      // g-band
    LMIC_setupChannel(4, 867300000, DR_RANGE_MAP(DR_SF12, DR_SF7),  BAND_CENTI);      // g-band
    LMIC_setupChannel(5, 867500000, DR_RANGE_MAP(DR_SF12, DR_SF7),  BAND_CENTI);      // g-band
    LMIC_setupChannel(6, 867700000, DR_RANGE_MAP(DR_SF12, DR_SF7),  BAND_CENTI);      // g-band
    LMIC_setupChannel(7, 867900000, DR_RANGE_MAP(DR_SF12, DR_SF7),  BAND_CENTI);      // g-band
    LMIC_setupChannel(8, 868800000, DR_RANGE_MAP(DR_FSK,  DR_FSK),  BAND_MILLI);      // g2-band
    
    // Disable link check validation
    LMIC_setLinkCheckMode(0);

    // Set data rate and transmit power (note: txpow seems to be ignored by the library)
    LMIC_setDrTxpow(DR_SF7,14);

    // Start job
    do_send(&sendjob);
}

void loop() {
      
    os_runloop_once

();
}
